In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
from pyvis.network import Network
import numpy as np

# Load your CSV
df = pd.read_csv('refined_per_data.csv')

# filter weights <= 10 but save the list of inlfuencers
influencers = [
    'ibai',
    'auronplay',
    'marta díaz',
    'djmario',
    'clakovi',
    'elrubius',
    'emma chamberlain',
    'malbert',
    'mostopapi',
    'orslok',
    'vegetta'
]

df = df[(df['Weight'] > 1 ) | (df['Target'].isin(influencers))]


# Determine the 10th and 90th percentile weights
lower_bound = np.percentile(df['Weight'], 10)
upper_bound = np.percentile(df['Weight'], 90)

# Calculate cumulative weight for each target node
node_weights = df.groupby('Target')['Weight'].sum().to_dict()

# Normalize node sizes for visualization
min_node_size = 10
max_node_size = 50
node_sizes = {
    node: np.interp(weight, (min(node_weights.values()), max(node_weights.values())), (min_node_size, max_node_size))
    for node, weight in node_weights.items()
}

# Create a pyvis network
net = Network(notebook=True)

# Add nodes with sizes based on cumulative weights
for node in pd.concat([df['Source'], df['Target']]).unique():  # Ensure every unique node is added
    size = node_sizes.get(node, min_node_size)  # If a node doesn't have a cumulative weight, use the minimum size
    net.add_node(node, size=size)

# Add edges with widths based on individual weights
for index, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['Weight']
    
    # Normalize edge width
    width = np.interp(weight, (lower_bound, upper_bound), (0.5, 5))
    
    net.add_edge(source, target, width=width)

# Display the network
net.show("network.html")




network.html


In [3]:
from IPython.display import display

# Sort the dataframe by 'Weight' in descending order
sorted_df = df.sort_values(by='Weight', ascending=False)

# Use pandas styling to highlight the 'Weight' column with a gradient and display the top rows 
display(sorted_df.head(20).style.background_gradient(cmap='Blues', subset=['Weight']))


,Source,Target,Weight
17,YouTuber,clakovi,379
22,YouTuber,clarakongvila,59
20,YouTuber,clara kong vila,12
9,YouTuber,brandy melville,6
63,YouTuber,tyler,6
2,YouTuber,alex.scalbert,2
4,YouTuber,augustana,2
15,YouTuber,carolina herrera,2
18,YouTuber,clara,2
26,YouTuber,david hockney,2


In [4]:
# Filter the DataFrame to only show rows corresponding to the influencers
filtered_df = df[df['Target'].isin(influencers)]

# Sort by weight in descending order
filtered_df = filtered_df.sort_values(by='Weight', ascending=False)

# Display the table
display(filtered_df.style.background_gradient(cmap='Blues', subset=['Weight']))

,Source,Target,Weight
17,YouTuber,clakovi,379
65,YouTuber,vegetta,2


In [5]:
# Convert the 'Source' column to the first name of the 'Target' column
filtered_df['Source'] = filtered_df['Target'].iloc[0]

# Remove the first row
filtered_df = filtered_df.iloc[1:]

print(filtered_df)

# Export the new DataFrame to CSV
filtered_df.to_csv('PER_filtered_data.csv', index=False)

     Source   Target  Weight
65  clakovi  vegetta       2
